# Prolific API basic walkthrough
Documentation: https://docs.prolific.co/docs/api-docs/public/

## Setup notebook

In [1]:
# import libraries
import json
import time
import urllib
import requests
import pandas as pd
import pymongo as pm
from IPython.display import clear_output

In [2]:
# define session wrapper to conveniently access API
class SessionWithUrlBase(requests.Session):
    def __init__(self, url_base=None, *args, **kwargs):
        super(SessionWithUrlBase, self).__init__(*args, **kwargs)
        self.url_base = url_base

    def request(self, method, url, **kwargs):
        modified_url = urllib.parse.urljoin(self.url_base, url)

        return super(SessionWithUrlBase, self).request(method, modified_url, **kwargs)

In [11]:
# get prolific credentials
tok = pd.read_csv('prolificToken.txt', header = None).values[0][0]

# create session
s = SessionWithUrlBase(url_base='https://api.prolific.co/api/v1/')
s.headers.update({
    'Authorization': f'Token {tok}',
    'Content-Type': 'application/json'
})

### Use case 1: Create a new draft Prolific Study
To create a new study, we execute a POST request to `studies/`, specifying the following study parameters in the POST call. The parameter descriptions can be found [here](https://docs.prolific.co/docs/api-docs/public/#tag/Studies/paths/~1api~1v1~1studies~1/post).

The documentation for specifying participant eligibility is incomplete, but you can modify `reqs` to modify the eligibility requirements

In [4]:
reqs = s.get('eligibility-requirements/').json()
reqs = reqs['results']

# it seems like
for study in reqs[221]['attributes']:
    study['value'] = True

In [5]:
# creates new study on Prolific
my_study = s.post('studies/',
      data=json.dumps({
          'name': 'BACH test study',
          'internal_name': 'test_study_0',
          'description': 'I think it works.',
          'external_study_url': 'https://cogtoolslab.org:8864/index.html?' +
          'PROLIFIC_PID={{%PROLIFIC_PID%}}&STUDY_ID={{%STUDY_ID%}}&SESSION_ID={{%SESSION_ID%}}',
          'prolific_id_option': 'url_parameters',
          'completion_code': 'BACH_completion_code',
          'completion_option': 'url',
          'total_available_places': 1,
          'estimated_completion_time': 10,
          'device_compatibility': ['desktop'],
          'reward': 200,
          'peripheral_requirements': [],
          'eligibility_requirements': reqs
      })
).json()

### Use case 2: Get information about a specific study

In [6]:
# display information about a specific study
study_inf = s.get(f"studies/{my_study['id']}/").json()
study_inf['total_available_places']

1

### Use case 3: Update study parameters

In [7]:
# Update study parameters. In this case, we increment study places by one
my_study = s.patch(f"studies/{my_study['id']}/",
        data=json.dumps({
            'total_available_places': my_study['total_available_places'] + 1,
        })
).json()
my_study['total_available_places']

2

### Use case 4: Stagger participant start times

Two primary benefits of publishing a study with this approach:
- Lower risk of race conditions where participants start the study before the DB updates (rare)
- Lower risk of race conditions where multiple participants accept the study before Prolific Gui updates (common)

In [8]:
# do we really want to publish the study and start staggering?
runStudy = False

# first, we find the current studies on Prolific
prolific_studies = pd.DataFrame(s.get(f"studies/").json()['results'])

# get the specific study we are running
my_study = prolific_studies.iloc[0]
my_study = s.get(f"studies/{my_study.id}/").json()
my_study['name']

'BACH test study'

In [9]:
num_participants = 50
stagger_time = 1 # seconds

# publish study
reallyRunStudy = False
if runStudy & reallyRunStudy:
    s.post(f"studies/{my_study['id']}/transition/",
          data=json.dumps({
              'action': "PUBLISH"
          })
    )

# opens a new study slot every stagger_time seconds
while my_study['total_available_places'] < num_participants:
    time.sleep(stagger_time)
    my_study = s.patch(f"studies/{my_study['id']}/",
                       data=json.dumps({
                           'total_available_places': my_study['total_available_places'] + 1,
                       })).json()
    print(f"Initializing study {my_study['name']} | {my_study['total_available_places']} places avaliable")
    clear_output(wait=True)


Initializing study BACH test study | 50 places avaliable


### Use case 5: Listing study submissions

In [21]:
my_study = prolific_studies[prolific_studies.status == 'COMPLETED'].iloc[0]
study_submissions = pd.DataFrame(s.get(f"studies/{my_study['id']}/submissions/").json()['results'])
study_submissions.head()

,id,participant_id,started_at,completed_at,is_complete,time_taken,reward,status,strata,study_code,star_awarded,bonus_payments,ip
0,626065dbeba3736512b6413f,61150ec9ff89021c26dcd061,2022-04-20T19:58:19.513000Z,2022-04-20T20:16:04.540000Z,True,1065.0,37593,APPROVED,"{'age': '28-37', 'ethnicity': 'white', 'sex': ...",5C772792,False,[],81.98.182.104
1,6260666186c85ad5dc06ecaf,6012b1a15107830438ca04a5,2022-04-20T20:00:37.220000Z,2022-04-20T20:16:04.284000Z,True,927.0,37593,APPROVED,"{'age': '38-47', 'ethnicity': 'white', 'sex': ...",5C772792,False,[],82.39.195.208
2,626066cd5987a7f950afd897,5afc4671bb36f900018ef1fe,2022-04-20T20:02:41.647000Z,2022-04-20T20:19:24.640000Z,True,1002.0,37593,APPROVED,"{'age': '28-37', 'ethnicity': 'white', 'sex': ...",5C772792,False,[],84.65.18.189
3,626066e9d5cfc3ea41421225,5e4721a7eab94f1f27d57fe0,2022-04-20T20:02:55.797000Z,None,False,NaN,37593,RETURNED,"{'age': '48-57', 'ethnicity': 'asian', 'sex': ...",None,False,[],77.99.112.90
4,626068786164bd067347d80c,5f9932d4ae84ec000947a76c,2022-04-20T20:09:28.835000Z,None,False,NaN,37593,RETURNED,"{'age': '18-27', 'ethnicity': 'mixed', 'sex': ...",None,False,[],86.147.75.224


### Use case 6: Applying bonus payments to participant submissions

In [47]:
reallyBonus = False

study_submissions['bonuses'] = np.round(np.random.random(6), 2)
submission_csv = study_submissions[['participant_id', 'bonuses']].to_csv(
    index=False, header=False, line_terminator='\n')
# example submission format
submission_csv

'61150ec9ff89021c26dcd061,0.55\n6012b1a15107830438ca04a5,0.07\n5afc4671bb36f900018ef1fe,0.41\n5e4721a7eab94f1f27d57fe0,0.33\n5f9932d4ae84ec000947a76c,0.77\n599364c1fc37790001fa4e1d,0.26\n'

In [49]:
# note: submitting bonus payments though this API does not actually transfer money
if reallyBonus:
    s.post(f"submissions/bonus-payments/", 
           data={
               "study_id": my_study['id'],
               "csv_bonuses": submission_csv
           })

# to transfer money, we run bonus payments like so:
reallyReallyBonus = False
if reallyBonus & reallyReallyBonus:
    s.post(f"bulk-bonus-payments/{my_study['id']}/pay/")

### Use case 5: Delete study

In [10]:
# delete study
s.delete(f"studies/{my_study['id']}/")

<Response [204]>